In [ ]:
!pip install faiss-gpu
!pip install pytorch_metric_learning
!pip install transformers

In [ ]:
import faiss
from tqdm import tqdm
import numpy as np
import pickle as pkl
from collections import OrderedDict
from typing import Dict
from transformers import AutoModel, AutoTokenizer
from sklearn.decomposition import PCA
import sys
! cp -r /content/drive/MyDrive/sapBERT-DUT-cambridge/src /content/src
from src.model_wrapper import Model_Wrapper
import pandas as pd
import sklearn
from sklearn.model_selection import ParameterGrid
import re

In [ ]:
finetunesets = {
    'nl-wiki_bel-test': 'nl_wiki_bel_traintestsplit',
    'mantra_both-files_cleaned_multiple_cuis' : 'nl_wiki_bel_all'
}

In [ ]:
sys.path.append('/content/drive/MyDrive/sapBERT-DUT-cambridge')
sapBERT_epochs = 3
finetune_epochs = 3
test_dataset = 'mantra_both-files_cleaned_multiple_cuis'
model_directory_path = f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa/ft_{finetune_epochs}_epoch/{finetunesets[test_dataset]}'

data = pd.read_pickle(f"/content/drive/MyDrive/sapBERT-DUT-cambridge/evaluation/{test_dataset}.pkl")
sentences = data['sentence'].to_list()
mentions = data['mention'].to_list()

trues = data['cui'].to_list()


In [ ]:
idx2cui = pkl.load(open('/content/drive/MyDrive/sapBERT-DUT-cambridge/index/id2cui.pkl', 'rb'))
cui2string = pkl.load(open('/content/drive/MyDrive/sapBERT-DUT-cambridge/index/cui_to_string', 'rb'))
relations = pkl.load(open('/content/drive/MyDrive/sapBERT_resources/relations', 'rb'))

In [ ]:
from src.model_wrapper import (
    Model_Wrapper
)

In [ ]:
def get_query_embedding(queries, tokenizer, model):
    bs = 128
    all_reps = []
    for i in tqdm(np.arange(0, len(queries), bs)):
        toks = tokenizer.batch_encode_plus(queries[i:i+bs],
                                        padding="max_length",
                                        max_length=25,
                                        truncation=True,
                                        return_tensors="pt")
        toks_cuda = {}
        for k,v in toks.items():
            toks_cuda[k] = v.cuda()
        output = model(**toks_cuda)

        cls_rep = output[0][:,0,:]

        all_reps.append(cls_rep.cpu().detach().numpy())
    all_reps_emb = np.concatenate(all_reps, axis=0)

    return all_reps_emb

def query_index(queries, tokenizer, model, index, idx2cui, cui2string, pca):
    query_embs = get_query_embedding(queries, tokenizer, model)
    query_embs = np.array(pca.transform(query_embs), dtype=np.float32)

    preds = []

    for query_emb in tqdm(query_embs):
        dist, neighbors = index.search(np.reshape(query_emb, (1,256)), 5)
        dist, neighbors = dist[0], neighbors[0]

        pred = idx2cui[neighbors[0]]
        preds.append(pred)

    return preds

In [ ]:
def getResources(model_directory_path):
    # if basemodel
    if '/medRoBERTa/ft_0_epoch/' in model_directory_path:
        tokenizer = AutoTokenizer.from_pretrained('CLTL/MedRoBERTa.nl')
        model = AutoModel.from_pretrained('CLTL/MedRoBERTa.nl').to('cuda')

        index = faiss.read_index('/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa/ft_0_epoch/index')
        pca = pkl.load(open('/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa/ft_0_epoch/pca', "rb"))

        # index = faiss.read_index('/content/drive/MyDrive/sapBERT-DUT-cambridge/index/index_basemodel_pca')
        # pca = pkl.load(open('/content/drive/MyDrive/sapBERT-DUT-cambridge/basemodel_pca', "rb"))
    else:
        model_wrapper = Model_Wrapper().load_model(
                path=f'{model_directory_path}',
                # path='/content/drive/MyDrive/sapBERT-DUT-cambridge/output_finetune_3epoch',
                max_length=25,
                use_cuda=True,
            )
        tokenizer = model_wrapper.get_dense_tokenizer()
        model = model_wrapper.get_dense_encoder()
        index = faiss.read_index(f'{model_directory_path}/index')
        pca = pkl.load(open(f'{model_directory_path}/pca', "rb"))
        # index = faiss.read_index('/content/drive/MyDrive/sapBERT-DUT-cambridge/finetuned_3epoch/index_pca')
        # pca = pkl.load(open('/content/drive/MyDrive/sapBERT-DUT-cambridge/finetuned_3epoch/pca', "rb"))
    return tokenizer, model, index, pca

In [ ]:
def checkRelations(pred, trues, relations):
    if isinstance(trues, list):
        for true in trues:
            key = f"{true}|{pred}"
            if key in relations:
                return relations[key]
    else:
        key = f"{trues}|{pred}"
        key2 = f"{pred}|{trues}"
        if key in relations:
            return relations[key]
        elif key2 in relations:
            return relations[key2]

    return False

In [ ]:
def checkRelation(preds, trues, relations):
    trues_ = []
    relas = []
    dist_1_relas = []

    for i, pred in enumerate(preds):

        rela = checkRelations(pred, trues[i], relations)
        # current_trues = re.findall('C\d{7}', trues[i])
        current_trues = trues[i]

        if pred in current_trues:
            trues_.append(pred)
            dist_1_relas.append(True)
            relas.append('')
        elif rela:
            trues_.append(current_trues[0])
            dist_1_relas.append(True)
            relas.append(rela)
            # relas.append(True)
        else:
            trues_.append(current_trues[0])
            dist_1_relas.append(False)
            relas.append('')
    return trues_, dist_1_relas, relas

In [ ]:
def predict(model_directory_path, sentences, mentions, idx2cui, cui2string):
    tokenizer, model, index, pca = getResources(model_directory_path)
    preds = query_index(mentions, tokenizer, model, index, idx2cui, cui2string, pca)
    return preds

In [ ]:
def evaluate(preds, trues, relas):
    acc = sklearn.metrics.accuracy_score(checked_trues, preds)
    acc_1dist = sum(relas)/len(relas)

    print(f"Accuracy: {acc}")
    print('1-dist accuracy: ', {acc_1dist})

    return acc, acc_1dist

In [ ]:
def saveResults(sentences, mentions, cui2string, preds, trues, relas, save=False):
    results = []

    for i, true in enumerate(trues):
        try:
            results.append([sentences[i], mentions[i], cui2string[preds[i]], cui2string[true], preds[i], true, relas[i]])
        except:
            print(mentions[i], true, preds[i])
            # continue
    df_results = pd.DataFrame(results, columns=['sentence', 'mention', 'prediction', 'label', 'cui_prediction', 'cui_label', 'relation'])

    # display(df_results)

    if save:
        df_results.to_csv(f'{model_directory_path}/predictions_multi-cuis-mantra.csv')

        with open(f'{model_directory_path}/preds', 'wb') as f:
            pkl.dump(preds, f)

In [ ]:
results = []

param_grid = [
    {
    '2nd_sapBERT_epochs' : 0,
    '3rd_finetune_epochs' : 0,
    'testsets' : 'mantra_both-files_cleaned_multiple_cuis'
    },
     {
    '2nd_sapBERT_epochs' : 3,
    '3rd_finetune_epochs' : 3,
    'testsets' : 'mantra_both-files_cleaned_multiple_cuis'
     }
]


for c in param_grid:
    print(c)
    test_dataset = c['testsets']
    finetune_epochs = c['3rd_finetune_epochs']
    sapBERT_epochs = c['2nd_sapBERT_epochs']

    # if basemodel
    if c['2nd_sapBERT_epochs'] == 0 and c['3rd_finetune_epochs'] == 0:
        model_directory_path = f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa/ft_0_epoch/{finetunesets[test_dataset]}'
    # if no 2nd phase pretraining
    elif c['2nd_sapBERT_epochs'] == 0:
        model_directory_path = f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa/ft_{finetune_epochs}_epoch/{finetunesets[test_dataset]}'
    # if no finetuning
    elif c['3rd_finetune_epochs'] == 0:
        model_directory_path = f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa_sapBERT/{sapBERT_epochs}_epoch/ft_0_epoch'
    else:
        model_directory_path = f'/content/drive/MyDrive/sapBERT-DUT-cambridge/results/medRoBERTa_sapBERT/{sapBERT_epochs}_epoch/ft_{finetune_epochs}_epoch/{finetunesets[test_dataset]}'

    data = pd.read_pickle(f"/content/drive/MyDrive/sapBERT-DUT-cambridge/evaluation/{test_dataset}.pkl")
    sentences = data['sentence'].to_list()
    mentions = data['mention'].to_list()
    if test_dataset == 'nl-wiki_bel-test':
        trues = data['checked_cuis'].to_list()
    else:
        trues = data['cui'].to_list()

    preds = predict(model_directory_path, sentences, mentions, idx2cui, cui2string)
    checked_trues, dist_1_relas, relas = checkRelation(preds, trues, relations)
    acc, acc_1dist = evaluate(preds, checked_trues, dist_1_relas)
    saveResults(sentences, mentions, cui2string, preds, checked_trues, relas, True)

    results.append([c['2nd_sapBERT_epochs'], c['3rd_finetune_epochs'], c['testsets'], acc, acc_1dist])


In [ ]:
df_results = pd.DataFrame(results, columns=['sapBERT_epochs', 'finetune_epochs', 'test_set', 'acc', '1-dist_acc'])
df_results

In [ ]:
df_results.to_csv('/content/drive/MyDrive/sapBERT-DUT-cambridge/evaluation/results_mantra-both-files_cleaned_multiple-cuis.csv')

In [ ]:
df_results.to_csv('results_mantra_both-files.csv')

In [ ]:
data